# Blockchain-Tutorial

## Imports / configuration

In [1]:
import hashlib
import binascii
import datetime
import collections

Import der spezifischen Krypto-Funktionen

In [2]:
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

## Funktionen

### Verschiedene Hilfsfunktionen

In [3]:
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

In [4]:
def display_transaction(transaction):
    #for transaction in transactions:
    dict = transaction.to_dict()
    print ("sender: " + dict['sender'])
    print ('-----')
    print ("recipient: " + dict['recipient'])
    print ('-----')
    print ("value: " + str(dict['value']))
    print ('-----')
    print ("time: " + str(dict['time']))
    print ('-----')



In [5]:
def dump_blockchain(Chain):
    print("Number of blocks in the chain: " + str(len(Chain)))
    block_temp = Chain[len(Chain)-1]
    print("Last block :")
    print("Root hash")
    print(block_temp.root_hash)
    print("--------------")
    print("Nonce")
    print(block_temp.Nonce)
    print("--------------")
    print("Previous block hash")
    print(block_temp.previous_block_hash)
    print("=====================================")

### Mining-Funktion

In [6]:
def mine(message, difficulty=1):
    assert difficulty >= 1
    prefix = '1' * difficulty
    for i in range(10000000):
        digest = sha256(str(hash(message)) + str(i))
        if digest.startswith(prefix):
            print ("after " + str(i) + " iterations found nonce: " + str(i) + ". Digest: " + str(digest))
            break
    return digest, i

### Klassen
Clients (wie in Server-Client). Kunden sind Instanzen mit:
- Einem Private-Key (wie z.B. das Smart-Card-Zertifikat)
- Einem Public-Key

In [8]:
class Client:
    def __init__(self):
        self._private_key = RSA.generate(1024, Crypto.Random.new().read)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

Klasse der Transaktionen. Jede Transaktion ist eine Instanz mit den folgenden Eigenschaften:
- Sender
- Empfänger
- Wert

In [9]:
class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()
    
    def to_ascii(self):
        my_str = ""
        # print(str(self.sender.identity))
        my_str = my_str + str(self.sender.identity)
        my_str = my_str + "&"
        # print(str(self.recipient))
        my_str = my_str + str(self.recipient)
        my_str = my_str + "&"
        # print(str(self.value))
        my_str = my_str + str(self.value)
        my_str = my_str + "&"
        # print(str(self.time))
        my_str = my_str + str(self.time)
        my_str = my_str + "&&"
        return my_str
    
    # in case of genesis, use this function
    def to_ascii_genesis(self):
        my_str = ""
        # print(str(self.sender.identity))
        my_str = my_str + str(self.sender)
        my_str = my_str + "&"
        # print(str(self.recipient))
        my_str = my_str + str(self.recipient)
        my_str = my_str + "&"
        # print(str(self.value))
        my_str = my_str + str(self.value)
        my_str = my_str + "&"
        # print(str(self.time))
        my_str = my_str + str(self.time)
        my_str = my_str + "&&"
        return my_str
    
    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time' : self.time})

Ausserdem gibt es die sogenannten Blocks, die alle Transaktionen aufnehmen. Blöcke beinhalten folgende Information:
- Root-Hash
- Hash-Wert des vorherigen Blocks
- Nonce-Wert

In [10]:
class Block:
    def __init__(self):
        self.root_hash = ""
        self.previous_block_hash = ""
        self.Nonce = ""
        
    def to_ascii(self):
        my_str = ""
        # print(str(self.root_hash))
        my_str = my_str + str(self.root_hash)
        my_str = my_str + "&"
        # print(str(self.previous_block_hash))
        my_str = my_str + str(self.previous_block_hash)
        my_str = my_str + "&"
        # print(str(self.Nonce))
        my_str = my_str + str(self.Nonce)
        my_str = my_str + "&&"
        return my_str

## Code

Globale Variabeln initialisieren
- Unser Blockchain (= eine Liste von Blocks)
- Liste mit Transaktionen für später
- Der Hash-Wert des letzten Blocks

In [11]:
BlockChain = []
transactions = []
last_block_hash = ""

### Clients schaffen

In [12]:
# clients
Benjamin = Client()        
Franziska = Client()
Stephanie = Client()
Steven = Client()

### Erste Transaktion

In [13]:
# initial transaction
t0 = Transaction ("Genesis", Benjamin.identity, 500.0)

In [14]:
display_transaction(t0)

sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100bd166aff950978bad6e1ed5f0d759365a003232ed28abed89b3103099b5add02ab0a3742f8099abc6790e6fd3bf1d2bbe6553982c19ae90d3cb07378df27426186cfc69488cf9787bbdf0396d87306a624023f36b4e411150a80f38c31a7e55ff5f0d9da05d8c6ee2e8464ad442cc0e45965c7c87b2a7611e63d79d64570db8b0203010001
-----
value: 500.0
-----
time: 2020-08-11 17:47:27.129197
-----


### Block bilden

In [15]:
# create first block
block0 = Block()
block0.previous_block_hash = None
Nonce = None
block0.root_hash = sha256(t0.to_ascii_genesis())
digest = hash(block0.to_ascii())
last_block_hash = digest

# add the block to the chain
BlockChain.append(block0)

In [ ]:
print(digest)

### Transaktionen hinzufügen

Beispiele für Transaktionen

In [16]:
# Transaktion 0
transactions.append(Transaction(Benjamin, Franziska.identity, 5.0))
# Transaktion 1<
transactions.append(Transaction(Benjamin, Stephanie.identity, 6.0))
# Transaktion 2
transactions.append(Transaction(Franziska, Steven.identity, 2.0))
# Transaktion 3
transactions.append(Transaction(Stephanie, Franziska.identity, 4.0))
# Transaktion 4
transactions.append(Transaction(Steven, Stephanie.identity, 7.0))
# Transaktion 5
transactions.append(Transaction(Franziska, Stephanie.identity, 3.0))
# Transaktion 6
transactions.append(Transaction(Stephanie, Benjamin.identity, 8.0))
# Transaktion 7
transactions.append(Transaction(Stephanie, Franziska.identity, 1.0))

In [ ]:
display_transaction(transactions[7])

### Weiteren Block bilden

In [17]:
block = Block()

Transaktionen 0 bis 4 "hashen"

In [18]:
hash_0 = sha256(transactions[0].to_ascii())
hash_1 = sha256(transactions[1].to_ascii())
hash_2 = sha256(transactions[2].to_ascii())
hash_3 = sha256(transactions[3].to_ascii())

# Merkle Tree
hash_01 = sha256(str(hash_0)+str(hash_1))
hash_23 = sha256(str(hash_2)+str(hash_3))
hash_0123 = sha256(str(hash_01)+str(hash_23))

print(hash_01)

156415084cf228388105ce9560a4ca59d9ba93189b16c7bc1b31e201400a887e


Und dem Block hinzufügen...

In [19]:
block.previous_block_hash = last_block_hash
block.root_hash = hash_0123

print(block.root_hash)

b4880ab3eafce7aa38d1d3f3560d881dab93d40f3c26c679fdd0368e61aea60d


Mining

In [20]:
digest, block.Nonce = mine(block, 5)

after 452920 iterations found nonce: 452920. Digest: 11111af5b092ba8478f02652acca812306fb34187ec334e0b8168e3d14747dab


In [21]:
BlockChain.append(block)
last_block_hash = digest

Wie sieht unsere Blockchain nun aus?

In [22]:
dump_blockchain(BlockChain)

Number of blocks in the chain: 2
Last block :
Root hash
b4880ab3eafce7aa38d1d3f3560d881dab93d40f3c26c679fdd0368e61aea60d
--------------
Nonce
452920
--------------
Previous block hash
1299605300851233674


## Überprüfen einer Transaktion

In [23]:
my_transaction = Transaction(Franziska, Steven.identity, 2.0)
my_transaction.time = transactions[2].time

display_transaction(my_transaction)

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100a76d5ecd50477586e29412f0e3a5185072433f46d5d28971375ed2dc9e31ba3f6ea70115c016654fc3cecab0426916ceba4ac5c8858fedc29cd9c8210f31951a20c75943f59a4b6c0bfd5ca88e6b3626ac9d0d85f2ffde984117f40c71b4fe81464b0b388fca5532d11d285a586eff819853ab916426713fafb6ee58a0fbf8170203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d00308189028181009bc897caa47cbcc5fb943bccaa97fb66f65746ffcb79c0c34b8c65bdfee144e1231a49f1e4e2d5346b0c214ecd0a1f0e088f72800d7dd1fb53810df4b831efaca6a6237927ad86ac5f6f76fefadf4d21eced593bb2c9f42e6d994614f02d79e7d53cdbd0922fda390b964c15b577b19f781338a0180e82f7861f50dbd33b83850203010001
-----
value: 2.0
-----
time: 2020-08-11 17:47:43.695213
-----


In [24]:
hash_2 = sha256(my_transaction.to_ascii())

hash_23 = sha256(str(hash_2)+str(hash_3))
hash_0123 = sha256(str(hash_01)+str(hash_23))

print(hash_0123)

b4880ab3eafce7aa38d1d3f3560d881dab93d40f3c26c679fdd0368e61aea60d


In [25]:
print(BlockChain[1].root_hash)

b4880ab3eafce7aa38d1d3f3560d881dab93d40f3c26c679fdd0368e61aea60d
